In [461]:
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [462]:
from __future__ import print_function

import keras
from keras import metrics
from keras.datasets import mnist
from keras.models import Sequential
from keras.optimizers import RMSprop, SGD
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv3D, MaxPooling3D

In [463]:
"""
Returns the tau's to be predicted
"""
def get_output_data():
    tau_11 = loadmat('tau11_xyz_T1.mat')['tau11']
    tau_12 = loadmat('tau12_xyz_T1.mat')['tau12']
    tau_13 = loadmat('tau13_xyz_T1.mat')['tau13']
    tau_22 = loadmat('tau22_xyz_T1.mat')['tau22']
    tau_23 = loadmat('tau23_xyz_T1.mat')['tau23']
    tau_33 = loadmat('tau33_xyz_T1.mat')['tau33']
    return tau_11, tau_12, tau_13, tau_22, tau_23, tau_33

In [75]:
"""
Returns the tau's to be predicted
"""
def get_input_data():
    uf = loadmat('u_F_xyz_T1.mat')['u_F']
    vf = loadmat('v_F_xyz_T1.mat')['v_F']
    wf = loadmat('w_F_xyz_T1.mat')['w_F']
    return uf, vf, wf

In [73]:
"""
Explores Dataset
"""
def explore_data(data):
    plt.figure(figsize=(15,5))
    # Varying by X
    plt.subplot(1,3,1)
    plt.plot(data[:,0,0])
    # Varying by Y
    plt.subplot(1,3,2)
    plt.plot(data[0,:,0])
    # Varying by Z
    plt.subplot(1,3,3)
    plt.plot(data[0,0,:])
    plt.show()

In [337]:
"""
Reshapes Data and split data into train and test sets
"""
def create_train_test_sets(tau_11, tau_12, tau_13, tau_22, tau_23, tau_33,
                           uf, vf, wf,
                           train_pct):
    x_dataset = np.transpose(np.array([uf.flatten(), vf.flatten(), wf.flatten()]))
    tau_11_dataset = np.transpose(np.array([tau_11.flatten()]))
    tau_12_dataset = np.transpose(np.array([tau_12.flatten()]))
    tau_13_dataset = np.transpose(np.array([tau_13.flatten()]))
    tau_22_dataset = np.transpose(np.array([tau_22.flatten()]))
    tau_23_dataset = np.transpose(np.array([tau_23.flatten()]))
    tau_33_dataset = np.transpose(np.array([tau_33.flatten()]))

    print(x_dataset.shape)
    print(tau_11_dataset.shape, tau_12_dataset.shape, tau_13_dataset.shape,
          tau_22_dataset.shape, tau_23_dataset.shape, tau_33_dataset.shape)

    train_test_index = (np.random.rand(x_dataset.shape[0]) < train_pct)

    x_train, x_test = x_dataset[train_test_index,:], x_dataset[~train_test_index,:]

    tau_11_train, tau_11_test = tau_11_dataset[train_test_index,:], tau_11_dataset[~train_test_index,:]
    tau_12_train, tau_12_test = tau_12_dataset[train_test_index,:], tau_12_dataset[~train_test_index,:]
    tau_13_train, tau_13_test = tau_13_dataset[train_test_index,:], tau_13_dataset[~train_test_index,:]
    tau_22_train, tau_22_test = tau_22_dataset[train_test_index,:], tau_22_dataset[~train_test_index,:]
    tau_23_train, tau_23_test = tau_23_dataset[train_test_index,:], tau_23_dataset[~train_test_index,:]
    tau_33_train, tau_33_test = tau_33_dataset[train_test_index,:], tau_33_dataset[~train_test_index,:]

    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    
    return (x_train, x_test, tau_11_train, tau_11_test, tau_12_train, tau_12_test, tau_13_train, tau_13_test,
           tau_22_train, tau_22_test, tau_23_train, tau_23_test, tau_33_train, tau_33_test)

In [506]:
"""
Reshapes Data and split data into train and test sets (Convolutional Set up)
"""
def create_train_test_sets_conv3d(tau_11, tau_12, tau_13, tau_22, tau_23, tau_33,
                                uf, vf, wf,
                                train_pct):
        
    train_index = np.concatenate((np.ones((int(np.floor(uf.shape[0]*train_pct)), uf.shape[1], uf.shape[2]), dtype = 'bool'),
                           np.zeros((int(np.ceil(uf.shape[0]*(1-train_pct))), uf.shape[1], uf.shape[2]), dtype = 'bool')),
                           axis = 0)

    print(train_index.shape)
    
    test_index = ~(train_index)
    
    train_index[0,:,:] = False
    train_index[uf.shape[0]-1,:,:] = False
    train_index[:,0,:] = False
    train_index[:,uf.shape[1]-1,:] = False
    train_index[:,:,0] = False
    train_index[:,:,uf.shape[2]-1] = False
    test_index[0,:,:] = False
    test_index[uf.shape[0]-1,:,:] = False
    test_index[:,0,:] = False
    test_index[:,uf.shape[1]-1,:] = False
    test_index[:,:,0] = False
    test_index[:,:,uf.shape[2]-1] = False
    
    train_locs = np.where(train_index)
    test_locs = np.where(test_index)
    
    print(train_locs)
    
    tau_11_train, tau_11_test = np.transpose([tau_11[train_locs]]), np.transpose([tau_11[test_locs]])
    tau_12_train, tau_12_test = np.transpose([tau_12[train_locs]]), np.transpose([tau_12[test_locs]])
    tau_13_train, tau_13_test = np.transpose([tau_13[train_locs]]), np.transpose([tau_13[test_locs]])
    tau_22_train, tau_22_test = np.transpose([tau_22[train_locs]]), np.transpose([tau_22[test_locs]])
    tau_23_train, tau_23_test = np.transpose([tau_23[train_locs]]), np.transpose([tau_23[test_locs]])
    tau_33_train, tau_33_test = np.transpose([tau_33[train_locs]]), np.transpose([tau_33[test_locs]])
    
    x_train = np.array([np.stack([uf[(x-1):(x+2),(y-1):(y+2),(z-1):(z+2)],
                                  vf[(x-1):(x+2),(y-1):(y+2),(z-1):(z+2)],
                                  wf[(x-1):(x+2),(y-1):(y+2),(z-1):(z+2)]], axis = 3)
              for x,y,z in zip(train_locs[0], train_locs[1], train_locs[2])])
    
    x_test = np.array([np.stack([uf[(x-1):(x+2),(y-1):(y+2),(z-1):(z+2)],
                                  vf[(x-1):(x+2),(y-1):(y+2),(z-1):(z+2)],
                                  wf[(x-1):(x+2),(y-1):(y+2),(z-1):(z+2)]], axis = 3)
              for x,y,z in zip(test_locs[0], test_locs[1], test_locs[2])])

    print('X_train shape', x_train.shape)
    print('tau_train shape', tau_11_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    
    return (x_train, x_test, tau_11_train, tau_11_test, tau_12_train, tau_12_test, tau_13_train, tau_13_test,
           tau_22_train, tau_22_test, tau_23_train, tau_23_test, tau_33_train, tau_33_test)

In [534]:
"""
Trains Simple One-Layer Neural Network with Relu Activation Functions
"""
def train_simple_nn_model(x_train, x_test, y_train, y_test, act_func = 'tanh',
                          batch_size = 20, epochs = 20, num_nodes = 10, xdim = 3):

    model = Sequential()
    model.add(Dense(num_nodes, activation=act_func, input_shape=(xdim,)))
    model.add(Dense(1, activation=act_func, input_shape=(xdim,)))

    model.summary()

    model.compile(loss='mean_squared_error',
                  optimizer=RMSprop(),
                  metrics=[metrics.mse])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model


In [547]:
"""
Trains Two-Layer Neural Network with Relu Activation Functions
"""
def train_two_layer_nn_model(x_train, x_test, y_train, y_test, act_func = 'tanh',
                             batch_size = 128, epochs = 20, num_nodes = (10,10), xdim = 3):

    model = Sequential()
    model.add(Dense(num_nodes[0], activation=act_func, input_shape=(xdim,)))
    model.add(Dense(num_nodes[1], activation=act_func))
    model.add(Dense(1, activation=act_func))

    model.summary()

    model.compile(loss='mean_squared_error',
                  optimizer=SGD(),
                  metrics=[metrics.mse])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model


In [518]:
"""
Trains Two-Layer Neural Network with Relu Activation Functions
"""
def train_conv_3d_model(x_train, x_test, y_train, y_test, act_func = 'tanh',
                          batch_size = 128, epochs = 20, num_nodes = 6, xdim = 3):

    model = Sequential()
    model.add(Conv3D(128, kernel_size = (3,3,3), data_format = 'channels_last',
                     input_shape = x_train.shape[1:], kernel_initializer = 'random_uniform'))
    model.add(Activation(act_func))
    model.add(Flatten())
    model.add(Dense(16, activation = act_func))
    model.add(Dense(1, activation = act_func))
    

    model.summary()

    model.compile(loss='mse',
                  optimizer=SGD(),
                  metrics=[metrics.mse])

    history = model.fit(x_train, y_train,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])
    
    return model


In [246]:
"""
Plots Actual vs. Predicted Values from Model
"""
def predict_and_visualize(model, x_test, y_test):
    y_predict = model.predict(x_test)
    sample_index = (np.random.rand(y_test.shape[0]) < 1000./y_test.shape[0])
    plt.figure(figsize=(15,5))
    plt.plot(y_test[sample_index])
    plt.plot(y_predict[sample_index])
    plt.show()

In [548]:
models = {'one-layer NN': (create_train_test_sets, train_simple_nn_model),
          'two-layer NN': (create_train_test_sets, train_two_layer_nn_model),
          'conv-3d NN': (create_train_test_sets_conv3d, train_conv_3d_model)}

In [406]:
"""
Plots Actual vs. Predicted Values from Model
"""
def predict_and_visualize(model, x_test, y_test):
    y_predict = model.predict(x_test)
    sample_index = (np.random.rand(y_test.shape[0]) < 1000./y_test.shape[0])
    plt.figure(figsize=(15,5))
    plt.plot(y_test[sample_index])
    plt.plot(y_predict[sample_index])
    plt.show()

In [521]:
"""
Plots Actual vs. Predicted Values from Model
"""
def predict(model, x_test, y_test):
    y_test = y_test.flatten()
    y_predict = model.predict(x_test).flatten()
    return np.corrcoef(y_test, y_predict), np.sqrt(((y_test - y_predict) ** 2).mean())

In [532]:
"""
Main Function to Execute Model
"""
def main(model_name):
    
    # Output Data
    tau_11, tau_12, tau_13, tau_22,tau_23, tau_33 = get_output_data()
    print('Shape of Output Files:')
    print(tau_11.shape, tau_12.shape, tau_13.shape, tau_22.shape, tau_23.shape, tau_33.shape)
    
    # Input Data
    uf, vf, wf = get_input_data()
    print('Shape of Input Files:')
    print(uf.shape, vf.shape, wf.shape)
    
    # Explore Data
    #explore_data(tau_12)
    
    # Get Functions
    train_test_split_func, model_func = models[model_name]
    
    # Reshape Data and Get Train/Test Sets
    (x_train, x_test, tau_11_train, 
    tau_11_test, tau_12_train, tau_12_test, tau_13_train, tau_13_test,
    tau_22_train, tau_22_test, tau_23_train, tau_23_test, 
    tau_33_train, tau_33_test) = train_test_split_func(tau_11, tau_12, tau_13, tau_22, tau_23, 
                                                       tau_33,uf, vf, wf, train_pct = 0.5)
    
    print(x_train[0,:])
    print(tau_11_train[0,:])
    
    # Train the Model
    models_final = [model_func(x_train, x_test, tau_11_train, tau_11_test, act_func = 'relu'),
                    model_func(x_train, x_test, tau_12_train, tau_12_test, act_func = 'tanh'),
                    model_func(x_train, x_test, tau_13_train, tau_13_test, act_func = 'tanh'),
                    model_func(x_train, x_test, tau_22_train, tau_22_test, act_func = 'relu'),
                    model_func(x_train, x_test, tau_23_train, tau_23_test, act_func = 'tanh'),
                    model_func(x_train, x_test, tau_33_train, tau_33_test, act_func = 'relu')]
    
    # Visualize Results
    results = [predict(models_final[0], x_test, tau_11_test),
               predict(models_final[1], x_test, tau_12_test),
               predict(models_final[2], x_test, tau_13_test),
               predict(models_final[3], x_test, tau_22_test),
               predict(models_final[4], x_test, tau_23_test),
               predict(models_final[5], x_test, tau_33_test)]
    
    return models_final, results
    

In [549]:
model_nn_1, results_nn_1 = main('one-layer NN')
model_nn_2, results_nn_2 = main('two-layer NN')
model_conv_3d, results_conv_3d = main('conv-3d NN')

Shape of Output Files:
(146, 96, 75) (146, 96, 75) (146, 96, 75) (146, 96, 75) (146, 96, 75) (146, 96, 75)
Shape of Input Files:
(146, 96, 75) (146, 96, 75) (146, 96, 75)
(1051200, 3)
(1051200, 1) (1051200, 1) (1051200, 1) (1051200, 1) (1051200, 1) (1051200, 1)
525435 train samples
525765 test samples
[ 0.61426491  0.35302837 -0.41810399]
[0.04341826]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_205 (Dense)            (None, 10)                40        
_________________________________________________________________
dense_206 (Dense)            (None, 10)                110       
_________________________________________________________________
dense_207 (Dense)            (None, 1)                 11        
Total params: 161
Trainable params: 161
Non-trainable params: 0
_________________________________________________________________
Train on 525435 samples, validate on 525765 samples
Ep

525435/525435 [==============================] - 9s 17us/step - loss: 9.1390e-05 - mean_squared_error: 9.1390e-05 - val_loss: 9.1748e-05 - val_mean_squared_error: 9.1748e-05
Epoch 15/20
525435/525435 [==============================] - 8s 16us/step - loss: 9.1090e-05 - mean_squared_error: 9.1090e-05 - val_loss: 9.1448e-05 - val_mean_squared_error: 9.1448e-05
Epoch 16/20
525435/525435 [==============================] - 9s 16us/step - loss: 9.0799e-05 - mean_squared_error: 9.0799e-05 - val_loss: 9.1170e-05 - val_mean_squared_error: 9.1170e-05
Epoch 17/20
525435/525435 [==============================] - 9s 17us/step - loss: 9.0530e-05 - mean_squared_error: 9.0530e-05 - val_loss: 9.0960e-05 - val_mean_squared_error: 9.0960e-05
Epoch 18/20
525435/525435 [==============================] - 9s 17us/step - loss: 9.0284e-05 - mean_squared_error: 9.0284e-05 - val_loss: 9.0725e-05 - val_mean_squared_error: 9.0725e-05
Epoch 19/20
525435/525435 [==============================] - 9s 18us/step - loss: 

Epoch 9/20
525435/525435 [==============================] - 9s 18us/step - loss: 9.3627e-04 - mean_squared_error: 9.3627e-04 - val_loss: 9.3206e-04 - val_mean_squared_error: 9.3206e-04
Epoch 10/20
525435/525435 [==============================] - 9s 18us/step - loss: 9.3618e-04 - mean_squared_error: 9.3618e-04 - val_loss: 9.3199e-04 - val_mean_squared_error: 9.3199e-04
Epoch 11/20
525435/525435 [==============================] - 9s 18us/step - loss: 9.3611e-04 - mean_squared_error: 9.3611e-04 - val_loss: 9.3192e-04 - val_mean_squared_error: 9.3192e-04
Epoch 12/20
525435/525435 [==============================] - 9s 17us/step - loss: 9.3605e-04 - mean_squared_error: 9.3605e-04 - val_loss: 9.3188e-04 - val_mean_squared_error: 9.3188e-04
Epoch 13/20
525435/525435 [==============================] - 8s 15us/step - loss: 9.3600e-04 - mean_squared_error: 9.3600e-04 - val_loss: 9.3184e-04 - val_mean_squared_error: 9.3184e-04
Epoch 14/20
525435/525435 [==============================] - 9s 17us/st

Epoch 4/20
525435/525435 [==============================] - 8s 16us/step - loss: 0.0024 - mean_squared_error: 0.0024 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 5/20
525435/525435 [==============================] - 8s 16us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 6/20
525435/525435 [==============================] - 8s 15us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 7/20
525435/525435 [==============================] - 8s 15us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 8/20
525435/525435 [==============================] - 8s 16us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_mean_squared_error: 0.0023
Epoch 9/20
525435/525435 [==============================] - 8s 16us/step - loss: 0.0023 - mean_squared_error: 0.0023 - val_loss: 0.0023 - val_mean_squared_

In [538]:
results_nn_1

[(array([[1.        , 0.66392491],
         [0.66392491, 1.        ]]), 0.019077221368526857),
 (array([[ 1.        , -0.01674562],
         [-0.01674562,  1.        ]]), 0.009428897983255155),
 (array([[1.        , 0.12001598],
         [0.12001598, 1.        ]]), 0.016034559925569295),
 (array([[1.        , 0.66578601],
         [0.66578601, 1.        ]]), 0.018875876372183832),
 (array([[1.        , 0.08084369],
         [0.08084369, 1.        ]]), 0.01591807086987144),
 (array([[1.        , 0.60346868],
         [0.60346868, 1.        ]]), 0.043272076305551896)]

In [550]:
results_nn_2

[(array([[1.00000000e+00, 8.58394439e-04],
         [8.58394439e-04, 1.00000000e+00]]), 0.030678746548232546),
 (array([[1.        , 0.00607609],
         [0.00607609, 1.        ]]), 0.009500571539990942),
 (array([[1.        , 0.10756245],
         [0.10756245, 1.        ]]), 0.01605620601876627),
 (array([[1.        , 0.00232454],
         [0.00232454, 1.        ]]), 0.030523476959390655),
 (array([[1.        , 0.04483455],
         [0.04483455, 1.        ]]), 0.016109418593734496),
 (array([[1.        , 0.50282927],
         [0.50282927, 1.        ]]), 0.046747823786944295)]

In [541]:
results_conv_3d

[(array([[1.        , 0.82924367],
         [0.82924367, 1.        ]]), 0.014053878766919289),
 (array([[1.       , 0.1616044],
         [0.1616044, 1.       ]]), 0.010007654023772862),
 (array([[1.        , 0.30826604],
         [0.30826604, 1.        ]]), 0.015735410668644874),
 (array([[1.       , 0.8364762],
         [0.8364762, 1.       ]]), 0.01379606574967823),
 (array([[1.        , 0.41828521],
         [0.41828521, 1.        ]]), 0.015750913407546335),
 (array([[1.        , 0.87821444],
         [0.87821444, 1.        ]]), 0.02630394266160002)]